Берем отызывы за лето (из архива с материалами или предыдущего занятия)

Учим conv сеть для классификации
Рассмотреть 2-а варианта сеточек
2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [55]:
!pip install pymorphy2

In [56]:
import numpy as np
import pandas as pd

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from pymorphy2 import MorphAnalyzer

import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, Flatten
from keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy

In [57]:
pip install stop_words

In [58]:
import re
import gensim
import numpy as np
import pandas as pd
from tqdm import tqdm
from string import punctuation
from functools import lru_cache

from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.callbacks import EarlyStopping

nltk.download("punkt")
tqdm.pandas()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [59]:
data = pd.read_excel('отзывы за лето.xls')
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [60]:
max_words = 20000
max_len = 150
num_classes = 5

epochs = 20
batch_size = 512
print_batch_n = 100

In [61]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [62]:
stopwordslist = stopwords.words("russian")
ptrn = r'[^a-zA-Zа-яА-Я0-9]'

morpher = MorphAnalyzer()

def words_only(text):
    text=str(text)
    return text.lower()

def remove_punkt(text):
    return re.sub(ptrn, ' ', text)

def to_token(text):
    return nltk.tokenize.word_tokenize(text)

def remove_stopwords(text):
    text_list = [w for w in text if w not in stopwordslist]
    return ' '.join(word for word in text_list)

def morphe_text(text):
    text = [morpher.parse(word)[0].normal_form for word in text.split() if word not in stopwordslist]
    return " ".join(text)

def normalize_text(text):
    text = words_only(text)
    text = remove_punkt(text)
    text = to_token(text)
    text = remove_stopwords(text)
    text = morphe_text(text)
    return text

In [63]:
data['normalized_content'] = data['Content'].apply(normalize_text)
data.head()

,Rating,Content,Date,normalized_content
0,5,It just works!,2017-08-14,it just works
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложение минус хотеть слишко...
2,5,Отлично все,2017-08-14,отлично
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать 1 работа антивирус далёкий нику...
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро


In [64]:
train_corpus = " ".join(data['normalized_content'])
train_tokens = word_tokenize(train_corpus)
train_tokens_filtered = [word for word in train_tokens if word.isalnum()]

In [65]:
dist = FreqDist(train_tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:7]

['приложение', 'очень', 'удобно', 'работать', 'удобный', 'вс', 'спасибо']

In [66]:
voc = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [67]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in voc:
            result.append(voc[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [68]:
data_train = np.asarray(
    [text_to_sequence(text, max_len) for text in data['normalized_content']],
    dtype=np.int32)

In [69]:
X_train, X_test, y_train, y_test = train_test_split(data_train, data.Rating, test_size=0.3, random_state=1)

In [70]:
X_train.shape

(14461, 150)

In [71]:
le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

le.classes_

array([1, 2, 3, 4, 5])

2.1 предобученные эмбединги

In [72]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))

model.add(GlobalMaxPool1D())
model.add(Flatten())

model.add(Dense(10))
model.add(Activation("relu"))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [73]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [74]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.1)

Epoch 1/10
26/26 [==============================] - 18s 579ms/step - loss: 1.1775 - accuracy: 0.7052 - val_loss: 0.9763 - val_accuracy: 0.7091
Epoch 2/10
26/26 [==============================] - 15s 565ms/step - loss: 0.8753 - accuracy: 0.7051 - val_loss: 0.7805 - val_accuracy: 0.7091
Epoch 3/10
26/26 [==============================] - 16s 608ms/step - loss: 0.7205 - accuracy: 0.7093 - val_loss: 0.7222 - val_accuracy: 0.7084
Epoch 4/10
26/26 [==============================] - 14s 559ms/step - loss: 0.6688 - accuracy: 0.7129 - val_loss: 0.7052 - val_accuracy: 0.7097
Epoch 5/10
26/26 [==============================] - 15s 592ms/step - loss: 0.6400 - accuracy: 0.7168 - val_loss: 0.6966 - val_accuracy: 0.7104
Epoch 6/10
26/26 [==============================] - 15s 574ms/step - loss: 0.6164 - accuracy: 0.7245 - val_loss: 0.6966 - val_accuracy: 0.7160
Epoch 7/10
26/26 [==============================] - 15s 582ms/step - loss: 0.5970 - accuracy: 0.7437 - val_loss: 0.6940 - val_accuracy: 0.7277

In [75]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 4s 300ms/step - loss: 0.7218 - accuracy: 0.7357
Test score: 0.7217864394187927
Test accuracy: 0.7357211709022522


Точность на предобученных эмбедингах ниже.